# EXR CONVERSION

In [ ]:
import sys
import array
import OpenEXR
import Imath
import numpy as np


In [ ]:
# Open the input file
file = OpenEXR.InputFile('/root/data/blender_test/Image0028_L.exr')

# Compute the size
dw = file.header()['dataWindow']
sz = (dw.max.x - dw.min.x + 1, dw.max.y - dw.min.y + 1)

# Read the three color channels as 32-bit floats
FLOAT = Imath.PixelType(Imath.PixelType.FLOAT)
(true_depth_map,G,B) = [array.array('f', file.channel(Chan, FLOAT)).tolist() for Chan in ("R", "G", "B") ]

In [ ]:
np.save('/root/data/blender_test/true_depth.npy', np.reshape(true_depth_map, (512, 1024)))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob

In [ ]:
left = sorted(glob.glob('/root/data/blender_v3/stereo_images/left*.png'))
right = sorted(glob.glob('/root/data/blender_v3/stereo_images/right*.png'))

# TAKE ONE RANDOM PAIR

In [ ]:
from utils import crop_and_mask, minimum_bounding_rectangle
import matplotlib.cm as cm



In [ ]:
# random_ind = np.random.randint(0, high=len(left))
# random_ind= 1101
# l = left[random_ind]
# r = right[random_ind]
# print(random_ind, left[random_ind])

In [ ]:
l = '/root/data/blender_test/Image0028_L.png'
r = '/root/data/blender_test/Image0028_R.png'

In [ ]:
# 3427

In [ ]:
f, ax = plt.subplots(1, 4, figsize=(20, 10))
ax[0].imshow(Image.open(l))
ax[1].imshow(Image.open(r))
ax[2].imshow(crop_and_mask(Image.open(l).convert('RGB')))
ax[3].imshow(crop_and_mask(Image.open(r).convert('RGB')))
plt.show()

In [ ]:
plt.imsave('test.png', crop_and_mask(Image.open(r)), cmap=cm.gray)

In [ ]:
mmm = np.array(Image.open('test.png').convert('L'))

In [ ]:
plt.imshow(mmm)

In [ ]:
mmm.max()

In [ ]:
mask_l = crop_and_mask(Image.open(l).convert('RGB'))
mask_r = crop_and_mask(Image.open(r).convert('RGB'))

In [ ]:
# left_corners
stack = np.stack([np.nonzero(mask_l)[0], np.nonzero(mask_l)[1]], axis=1)
left_corners = minimum_bounding_rectangle(stack)

In [ ]:
# right_corners
stack = np.stack([np.nonzero(mask_r)[0], np.nonzero(mask_r)[1]], axis=1)
right_corners = minimum_bounding_rectangle(stack)

In [ ]:
import matplotlib.patches as patches

In [ ]:
left_corners

In [ ]:
right_corners

In [ ]:
left_center = np.mean(left_corners[:, [1, 0]][1:3], axis=0)
right_center = np.mean(np.stack((left_corners[:, [1, 0]][0], left_corners[:, [1, 0]][-1]), axis=0), axis=0)

In [ ]:
np.stack((left_center, right_center))

In [ ]:
# Create figure and axes
fig,ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(Image.open(l))
rect = patches.Polygon(left_corners[:, [1, 0]], closed=True, linewidth=2,edgecolor='r',facecolor='none')
ax[0].add_patch(rect)
ax[0].add_patch(patches.Circle(left_center, radius=10, facecolor='b'))
ax[0].add_patch(patches.Circle(right_center, radius=10, facecolor='b'))
ax[0].plot((left_center[0], right_center[0]), (left_center[1], right_center[1]), color='b')

ax[1].imshow(Image.open(r))
rect = patches.Polygon(right_corners[:, [1, 0]], closed=True, linewidth=2,edgecolor='r',facecolor='none')
ax[1].add_patch(rect)
plt.show()

In [ ]:
# estimate diparity
disp = -8
# fig,ax = plt.subplots(2, 1, figsize=(20, 10))
shifted = np.zeros((512, 1024, 3))
shifted[:, disp:, :] = np.array(Image.open(l).convert('RGB'))[:, :-disp,:]
plt.imshow(shifted[200:350, 350:700])

plt.imshow(np.array(Image.open(r))[200:350, 350:700], alpha = 0.5)

In [ ]:
from PIL import Image, ImageDraw

In [ ]:
img = Image.new('L', (1024, 512), 0)
ImageDraw.Draw(img).polygon([tuple(t) for t in list(left_corners[:, [1, 0]])], outline=1, fill=1)
mask = np.array(img)

In [ ]:
plt.imshow(mask)
plt.show()

In [ ]:
# disparities
disp = right_corners[:,1] - left_corners[:,1]
print(disp)

In [ ]:
# # calculate coordinates of central point
# left_center_left = np.mean(left_corners[1:3], axis=0)
# left_center_right = np.mean([left_corners[0], left_corners[-1]], axis=0)
# right_center_left = np.mean(right_corners[1:3], axis=0)
# right_center_right = np.mean([right_corners[0], right_corners[-1]], axis=0)

In [ ]:
# # disparities
# left_center_disp= (right_center_left - left_center_left)[1]
# right_center_disp= (right_center_right - left_center_right)[1]

In [ ]:
# plt.plot([left_center_left[1], left_center_right[1]], [left_center_disp, right_center_disp])
# plt.show()

In [ ]:
from scipy.interpolate import interp2d

In [ ]:
# interpolation 2d
f = interp2d(left_corners[:,1], left_corners[:,0], disp)

In [ ]:
left_corners[:,0]

In [ ]:
left_corners[:,1]

In [ ]:
disp

In [ ]:
ynew = np.arange(0, 512, 1)
xnew = np.arange(0, 1024, 1)
znew = f(xnew, ynew)

In [ ]:
left_corners

In [ ]:
right_corners

In [ ]:
plt.imshow(znew)
plt.colorbar()
plt.show()

In [ ]:
# plt.imshow(np.array(Image.open(l)))
plt.imshow(znew*mask, alpha=0.7)
plt.colorbar()
plt.show()


In [ ]:
plt.imsave('depth.png', )

# CONVERT TO DEPTH

In [ ]:
disparities = znew*mask

In [ ]:
# meters
focal_length = 10.0*1e-3
baseline = 65.0*1e-3
image_sensor_width = 32.0*1e-3
image_sensor_height = 18.0*1e-3
pixel_size = image_sensor_width / 1024
print(pixel_size)

In [ ]:
focal_length_pixel = focal_length / pixel_size
print(focal_length_pixel)

In [ ]:
depth = focal_length_pixel*baseline / disparities

In [ ]:
focal_length_pixel*baseline / 3

In [ ]:
focal_length_pixel*baseline / 1280

In [ ]:
plt.imshow(depth)
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(np.load('/root/data/blender_test/true_depth.npy')[200:350, 350:700])
plt.colorbar()
plt.clim([6, 9])
plt.show()

In [ ]:
def convert_to_world_point(x, y, depth_map):
    image_center_x = depth_map.shape[1] / 2.0
    image_center_y = depth_map.shape[0] / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (image_sensor_width / depth_map.shape[1])
    sensor_z = px_z * (image_sensor_height / depth_map.shape[0])
    
    d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / focal_length
    world_z = (world_y * sensor_z) / focal_length
#     return world_y
    return (world_x, world_y, world_z)

In [ ]:
left_center

In [ ]:
world_left = convert_to_world_point(373, 268, depth)

In [ ]:
right_center

In [ ]:
world_right = convert_to_world_point(642, 288, depth)

In [ ]:
dist = np.linalg.norm(np.array(world_left) - np.array(world_right))

In [ ]:
print(dist)

In [ ]:
left_corners[:, [1, 0]]

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(20, 10))
ax.imshow(Image.open(l))
rect = patches.Polygon(left_corners[:, [1, 0]], closed=True, linewidth=2,edgecolor='r',facecolor='none')
ax.add_patch(rect)
ax.add_patch(patches.Circle(left_center, radius=10, facecolor='b'))
ax.add_patch(patches.Circle(right_center, radius=10, facecolor='b'))
ax.plot((left_center[0], right_center[0]), (left_center[1], right_center[1]), color='b')
ax.text(left_corners[:, [1, 0]][2,0], left_corners[:, [1, 0]][2,1]-10, str(dist)[:5] + 'm', color='b', size=15)
plt.show()